# 빅데이터 분석프로그래밍 2 - Week 06
카메라 기능과 cv2를 활용하여 얼굴 인식 프로그램을 만들어 보도록 하겠습니다.
____

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python
!pip install opencv-python

## 1. 얼굴인식 알고리즘 적용하기

In [ ]:
import matplotlib.pyplot as plt
import cv2

# 캐스케이드 파일 지정해서 검출기 생성하기 --- (*1)
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

# 이미지를 읽어 들이고 그레이스케일로 변환하기 --- (*2)
img = cv2.imread("face.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 얼굴 인식하기 --- (*3)
face_list = cascade.detectMultiScale(img_gray)


# 인식한 부분 표시하기 --- (*4)
x,y,w,h = face_list[0]
print("얼굴의 좌표 =", x, y, w, h)
red = (0, 0, 255)
cv2.rectangle(img, (x, y), (x+w, y+h), red, thickness=20)

# 이미지 출력하기
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

## Q1. 여러명의 얼굴을 인식 할 수 있게 위의 코드를 수정한다면?


In [ ]:
import matplotlib.pyplot as plt
import cv2

# 캐스케이드 파일 지정해서 검출기 생성하기 --- (*1)
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

# 이미지를 읽어 들이고 그레이스케일로 변환하기 --- (*2)
img = cv2.imread("faces.jpeg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 얼굴 인식하기 --- (*3)
face_list = cascade.detectMultiScale(img_gray)

##이부분을 수정해주세요!
##hint: 반복문


face = face_list[0]
# 인식한 부분 표시하기 --- (*4)
x,y,w,h = face
print("얼굴의 좌표 =", x, y, w, h)
red = (0, 0, 255)
cv2.rectangle(img, (x, y), (x+w, y+h), red, thickness=2)

# 이미지 출력하기
plt.figure(figsize=(20,10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

## 2. 얼굴에 모자이크 적용하기

In [ ]:
def mosaic(img, rect, size):
    # 모자이크 적용할 부분 추출하기
    (x1, y1, x2, y2) = rect
    w = x2 - x1
    h = y2 - y1
    i_rect = img[y1:y2, x1:x2]
    # 축소하고 확대하기
#     print(img.shape)
#     print(i_rect.shape)
#     plt.imshow(i_rect)
#     plt.show()
    i_small = cv2.resize(i_rect, ( size, size))
#     print(i_small.shape)
#     plt.imshow(i_small)
#     plt.show()
    i_mos = cv2.resize(i_small, (w, h), interpolation=cv2.INTER_AREA)

#     print(i_mos.shape)
#     plt.imshow(i_mos)
#     plt.show()
    # 모자이크 적용하기
    img2 = img.copy()
    img2[y1:y2, x1:x2] = i_mos
    return img2

In [ ]:
import matplotlib.pyplot as plt
import cv2


# 캐스케이드 파일 지정해서 검출기 생성하기 --- (*1)
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

# 이미지를 읽어 들이고 그레이스케일로 변환하기 --- (*2)
img = cv2.imread("face.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 얼굴 검출하기 --- (*3)
face_list = cascade.detectMultiScale(img_gray)
if len(face_list) == 0: 
    print("얼굴인식 실패")
else:
    # 인식한 부분에 모자이크 처리하기 --- (*4)
    for (x,y,w,h) in face_list:
        img = mosaic(img, (x, y, x+w, y+h), 10)

    # 이미지 출력하기
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

## 3. 파이썬에서 노트북 카메라 사용하기

In [ ]:
import numpy as np
import cv2

#현재 디바이스의 카메라를 호출하는 함수
cap = cv2.VideoCapture(0)

while(True):
    #카메라에서 반복적으로 현재 프레임 확보
    ret, frame = cap.read()
    if ret:
    #     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break
cap.release()
cv2.destroyAllWindows()

## 4. 카메라와 얼굴인식 결합하기

In [ ]:
def face_recognize(frame):
    # 캐스케이드 파일 지정해서 검출기 생성하기 --- (*1)
    cascade_file = "haarcascade_frontalface_alt.xml"
    cascade = cv2.CascadeClassifier(cascade_file)

    # 이미지를 읽어 들이고 그레이스케일로 변환하기 --- (*2)
    img = frame
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 얼굴 인식하기 --- (*3)
    face_list = cascade.detectMultiScale(img_gray)
    # 결과 확인하기 --- (*4)
    if len(face_list) == 0:
#         print("얼굴 인식 실패")
        return
    # 인식한 부분 표시하기 --- (*5)
    for (x,y,w,h) in face_list:
#         print("얼굴의 좌표 =", x, y, w, h)
        red = (0, 0, 255)
        cv2.rectangle(img, (x, y), (x+w, y+h), red, thickness=20)

    return img


In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    if ret:
        frame = face_recognize(frame)
        try:
            cv2.imshow('frame',frame)
        except:
            pass
        keyboard = cv2.waitKey(1)

        if keyboard & 0xFF == ord('q'):            
            break
    

cap.release()
cv2.destroyAllWindows()

## 5. 카메라와 모자이크 결합하기

In [ ]:
def face_mosaic(frame):
    # 캐스케이드 파일 지정해서 검출기 생성하기 --- (*1)
    cascade_file = "haarcascade_frontalface_alt.xml"
    cascade = cv2.CascadeClassifier(cascade_file)

    # 이미지를 읽어 들이고 그레이스케일로 변환하기 --- (*2)
    img = frame
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출하기 --- (*3)
    face_list = cascade.detectMultiScale(img_gray)
    if len(face_list) == 0: 
        return

    # 인식한 부분에 모자이크 처리하기 --- (*4)
    for (x,y,w,h) in face_list:
        img = mosaic(img, (x, y, x+w, y+h), 10)



    return img

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):

    ret, frame = cap.read()
    if ret:

        frame = face_mosaic(frame)
        try:
            cv2.imshow('frame',frame)
        except:
            pass
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


## References
- https://github.com/rintiantta/book-mlearn-2

In [ ]:
import sys
!{sys.executable} -m streamlit run app.py
#또는
# !streamlit run app.py